In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy.utils as fu

In [ ]:
irch_file = '../neversink_mf6/irch.dat'
id3_file = '../neversink_mf6/idomain_003.dat'
top_file = '../neversink_mf6/top.dat'

wkdir = '../neversink_mf6/'

In [ ]:
top = np.loadtxt(top_file)
top[top<-8000] = np.nan
plt.imshow(top)

plt.colorbar()

In [ ]:
id3  = np.loadtxt(id3_file, dtype=int)
plt.imshow(id3)

In [ ]:
irch = np.loadtxt(irch_file, dtype=int)
irch -= 1
plt.imshow(irch)
plt.colorbar()

In [ ]:
nrow, ncol = id3.shape
print(nrow, ncol)

In [ ]:
every_n_cells = 50

In [ ]:
j = list(range(ncol))[0:ncol:every_n_cells]
i = list(range(nrow))[0:nrow:every_n_cells]

In [ ]:
j

In [ ]:
i

In [ ]:
J,I = np.meshgrid(j,i)

In [ ]:
I

In [ ]:
J

In [ ]:
points = list(zip(I.ravel(),J.ravel()))

In [ ]:
keep_points = [(irch[i,j],i,j) for i,j in points if id3[i,j]==1]

In [ ]:
drop_points = [(0, 150, 50),(3, 150, 100),(3, 100, 50)]

In [ ]:
keep_points = [i for i in keep_points if i not in drop_points]

In [ ]:
len(keep_points)

In [ ]:
with open(os.path.join(wkdir,'land_surf_obs-indices.csv'), 'w') as ofp:
    ofp.write('k,i,j,obsname\n')
    [ofp.write('{0},{1},{2},land_surf_obs_{1}_{2}\n'.format(*i)) for i in keep_points]

### make an observations file

In [ ]:
with open(os.path.join(wkdir,'land_surf_obs-observations.csv'), 'w') as ofp:
    ofp.write('obsname,obsval\n')
    [ofp.write('land_surf_obs_{1}_{2},{3}\n'.format(*i, top[i[1],i[2]])) for i in keep_points]


## TEst out reading the HDS file

In [ ]:
h = fu.binaryfile.HeadFile(os.path.join(wkdir, 'neversink.hds')).get_data()

In [ ]:
ls_inds = pd.read_csv(os.path.join(wkdir,'land_surf_obs-indices.csv'))
ls_inds['obs'] = h[ls_inds.k, ls_inds.i, ls_inds.j]
ls_out = ls_inds[['obs']]
ls_out.index = ls_inds.obsname
ls_out